# ALGORITMO FILTRADO COLABORATIVO BASADO EN CONTENIDO

In [1]:
import pandas as pd
import numpy as np

import pickle

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

import joblib

In [2]:
df= pd.read_csv("./poblacion_adopcion_v3.csv")

In [3]:
df.head(10)

,Nombre,Sexo,Raza,Edad,Tamaño,Especie,id_adopcion,cat_abandonado,cat_cariño,cat_cercania,...,cat_independencia,cat_maltrato,cat_gato,cat_perro,cat_pelo,cat_macho,cat_hembra,cat_ladra,cat_ruido,id_centro
0,Tomás,Macho,Pastor Belga,Menos de 1 año,Mediano,Canina,399951,0.000000,0.065953,0.000000,...,0.0,0.0,0.000000,0.229896,0.00000,0.000000,0.0,0.227081,0.0,543
1,Carolina,Hembra,Mestizo,4 años,Mediano,Canina,313004,0.000000,0.022771,0.000000,...,0.0,0.0,0.000000,-0.102187,0.00000,0.000000,0.0,0.000000,0.0,118
2,Malvolio,Macho,Mestizo,5 años,Mediano,Canina,468,0.119279,0.073283,0.000000,...,0.0,0.0,0.000000,-0.115225,0.08198,0.000000,0.0,0.045177,0.0,757
3,Gabriela,Hembra,Mestizo,8 años,Mediano,Canina,2893,0.000000,0.126649,0.000000,...,0.0,0.0,-0.044153,-0.070310,0.00000,0.000000,0.0,0.000000,0.0,609
4,Citlali,Hembra,Podenco,5 años,Mediano,Canina,52357,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,-0.061876,0.00000,0.000000,0.0,0.000000,0.0,743
5,Alberto,Macho,Mestizo,1 años,Mediano,Canina,479250,0.042610,0.055986,0.000000,...,0.0,0.0,0.000000,-0.175857,0.00000,0.000000,0.0,0.000000,0.0,775
6,Bautista,Macho,Gato,2 años,Pequeño,Felina,394929,0.000000,0.000000,-0.071289,...,0.0,0.0,0.000000,-0.031498,0.00000,0.000000,0.0,0.000000,0.0,458
7,Santi,Macho,Gato,2 años,Pequeño,Felina,182913,0.000000,0.098665,0.000000,...,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,716
8,Santi,Macho,Cruce Podenco,2 años,Mediano,Canina,477844,0.000000,0.000000,0.073719,...,0.0,0.0,0.000000,-0.113126,0.00000,0.233825,0.0,0.000000,0.0,136
9,Karina,Hembra,Cruce Mastín,1 años,Mediano,Canina,320265,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,-0.206664,0.00000,0.000000,0.0,0.000000,0.0,553


Tratamiento de nulos

In [4]:
df.isnull().sum()

Nombre               0
Sexo                 0
Raza                 0
Edad                 0
Tamaño               0
Especie              0
id_adopcion          0
cat_abandonado       1
cat_cariño           1
cat_cercania         1
cat_acogida          1
cat_actividad        1
cat_dependencia      1
cat_adaptacion       1
cat_docil            1
cat_agradecido       1
cat_agresivo         1
cat_tenso            1
cat_alegre           1
cat_inteligencia     1
cat_miedo            1
cat_convivencia      1
cat_correa           1
cat_carretera        1
cat_aventura         1
cat_niños            1
cat_belleza          1
cat_ppp              1
cat_bruto            1
cat_triste           1
cat_tranquilidad     1
cat_desubicado       1
cat_recuperacion     1
cat_cronico          1
cat_historial        1
cat_fortaleza        1
cat_independencia    1
cat_maltrato         1
cat_gato             1
cat_perro            1
cat_pelo             1
cat_macho            1
cat_hembra           1
cat_ladra  

In [5]:
df= df.dropna()

Remuestreo para poder hacer el algoritmo (si no no me deja con tantos datos)

In [6]:
adopcion = df.sample(n=50000)

**División de datos en training-test**

In [7]:
X_train, X_test = train_test_split(adopcion, test_size=0.2, random_state=42)

**Selección de columnas (únicamente las puntuaciones)**

In [8]:
adopcion.columns

Index(['Nombre', 'Sexo', 'Raza', 'Edad', 'Tamaño', 'Especie', 'id_adopcion',
       'cat_abandonado', 'cat_cariño', 'cat_cercania', 'cat_acogida',
       'cat_actividad', 'cat_dependencia', 'cat_adaptacion', 'cat_docil',
       'cat_agradecido', 'cat_agresivo', 'cat_tenso', 'cat_alegre',
       'cat_inteligencia', 'cat_miedo', 'cat_convivencia', 'cat_correa',
       'cat_carretera', 'cat_aventura', 'cat_niños', 'cat_belleza', 'cat_ppp',
       'cat_bruto', 'cat_triste', 'cat_tranquilidad', 'cat_desubicado',
       'cat_recuperacion', 'cat_cronico', 'cat_historial', 'cat_fortaleza',
       'cat_independencia', 'cat_maltrato', 'cat_gato', 'cat_perro',
       'cat_pelo', 'cat_macho', 'cat_hembra', 'cat_ladra', 'cat_ruido',
       'id_centro'],
      dtype='object')

In [9]:
variables= ['cat_abandonado', 'cat_cariño', 'cat_cercania', 'cat_acogida',
       'cat_actividad', 'cat_dependencia', 'cat_adaptacion', 'cat_docil',
       'cat_agradecido', 'cat_agresivo', 'cat_tenso', 'cat_alegre',
       'cat_inteligencia', 'cat_miedo', 'cat_convivencia', 'cat_correa',
       'cat_carretera', 'cat_aventura', 'cat_niños', 'cat_belleza', 'cat_ppp',
       'cat_bruto', 'cat_triste', 'cat_tranquilidad', 'cat_desubicado',
       'cat_recuperacion', 'cat_cronico', 'cat_historial', 'cat_fortaleza',
       'cat_independencia', 'cat_maltrato', 'cat_gato', 'cat_perro',
       'cat_pelo', 'cat_macho', 'cat_hembra', 'cat_ladra', 'cat_ruido']

**Normalización de datos**

In [10]:
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train[variables])
X_test_norm = scaler.transform(X_test[variables])

**Cálculo de la matriz de similitud de coseno**

Las aplico sobre los datos de entrenamiento

In [11]:
item_sim_cos = cosine_similarity(X_train_norm)

**Entrenamiento del modelo**

In [12]:
filtrado = NearestNeighbors(metric='cosine', algorithm='brute')
filtrado.fit(X_train_norm)

NearestNeighbors(algorithm='brute', metric='cosine')

**Guardado del modelo**

In [13]:
#Guardo el modelo ya entrenado
with open('filtrado.pkl', 'wb') as archivo:
    pickle.dump(filtrado, archivo)

**Encontrar los 5 items más cercanos para el conjunto del test**

In [14]:
distancias, indices = filtrado.kneighbors(X_test_norm, n_neighbors=5)

In [21]:
recomendaciones= df.iloc[indices[0]]

In [22]:
recomendaciones

,Nombre,Sexo,Raza,Edad,Tamaño,Especie,id_adopcion,cat_abandonado,cat_cariño,cat_cercania,...,cat_independencia,cat_maltrato,cat_gato,cat_perro,cat_pelo,cat_macho,cat_hembra,cat_ladra,cat_ruido,id_centro
26951,Benjamín,Macho,Podenco,2 años,Mediano,Canina,234686,0.0,0.016532,0.000000,...,0.0,0.000000,0.0,-0.192996,0.000000,0.076489,0.000000,0.0,0.0,407
21055,Xalvador,Macho,Mestizo,1 años,Mediano,Canina,284844,0.0,0.000000,0.000000,...,0.0,0.021893,0.0,-0.060281,0.024398,0.000000,0.000000,0.0,0.0,901
33992,Jose,Macho,Bretón Español,2 años,Mediano,Canina,408683,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,-0.160328,0.000000,0.000000,0.000000,0.0,0.0,932
22940,Andrés,Macho,Cruce Pastor Belga,3 años,Mediano,Canina,15706,0.0,0.052126,0.000000,...,0.0,0.000000,0.0,-0.028029,0.000000,0.000000,-0.039304,0.0,0.0,558
32251,Luciano,Macho,Pastor Belga,7 años,Mediano,Canina,467473,0.0,0.149320,-0.189362,...,0.0,0.000000,0.0,-0.193713,0.000000,0.000000,0.000000,0.0,0.0,198


In [24]:
X_test_norm[0]

array([0.        , 0.15126377, 0.48756825, 0.        , 0.        ,
       0.27875952, 0.        , 0.11226232, 0.24123533, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.40778203,
       0.64652401, 0.        , 0.        , 0.63698701, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.11442361, 0.58923487, 0.6812308 , 0.        , 0.46204092,
       0.56463201, 0.30105599, 0.        ])

**Predecir resultados para el primer elemento del test**

In [15]:
predicciones = []
for i in range(len(X_test)):
    indices_fila = indices[i]
    distancias_fila = distancias[i]
    valoraciones_vecinos = X_train_norm[indices_fila]
    pesos = 1 - distancias_fila
    prediccion = np.dot(pesos, valoraciones_vecinos) / np.sum(pesos)
    predicciones.append(prediccion)


**Evaluación de resultados**

In [16]:
# ERROR CUADRÁTICO MEDIO (RMSE)
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(X_test_norm, predicciones))

In [17]:
rmse

0.026629829333551507

In [18]:
# ERROR ABSOLUTO MEDIO (MAE)
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(X_test_norm, predicciones)

In [19]:
mae

0.013068334471914333